In [30]:
import pandas as pd
import altair as alt

In [31]:
file_path = r"..\data\Main Data Compiled.xlsx"
df = pd.read_excel(file_path, sheet_name='Table 1.1')

df['Year'] = pd.to_numeric(df['Years'])
df = df[df['Year'] >= 1960]

df['Surplus'] = df['Surplus or Deficit (-)'].where(df['Surplus or Deficit (-)'] > 0, 0)    #ChatGPT prompt “how can I create separate ‘Surplus’ and ‘Deficit’ columns from a column that has positive and negative values?”
df['Deficit'] = df['Surplus or Deficit (-)'].where(df['Surplus or Deficit (-)'] < 0, 0)    # ChatGPT suggested np.where 

base = alt.Chart(df).encode(
    x=alt.X(
        'Year:O',
        title='Fiscal Year',
        axis=alt.Axis(
            labelAngle=90,
            labelFontSize=10,
            titleFontSize=12,
            labelExpr="datum.label % 2 == 0 ? datum.label : ''"
        )
    )
)



surplus_area = base.mark_area(color='#4CAF50', opacity=0.5).encode(
    y=alt.Y('Surplus:Q', title='Billions of U.S. Dollars'),
    y2=alt.datum(0),

)

deficit_area = base.mark_area(color='#E53935', opacity=0.5).encode(
    y='Deficit:Q',
    y2=alt.datum(0),

)

receipts_line = base.mark_line(color='#1E88E5', strokeWidth=2).encode(
    y='Receipts:Q',
)

outlays_line = base.mark_line(color='#FB8C00', strokeWidth=2).encode(
    y='Outlays:Q',
)

receipts_label = alt.Chart(df.tail(1)).mark_text(
    text='Receipts', align='left', dx=5, color='#1E88E5',               #ChatGPT prompt: "How can I add text labels (like 'Receipts' and 'Outlays') to the last points in an Altair chart?"
                                                                          #Chatgpt suggested mark_text
    fontSize=12, fontWeight='bold'                                  
).encode(x='Year:O', y='Receipts:Q')

outlays_label = receipts_label.encode(
    text=alt.value('Outlays'), color=alt.value('#FB8C00'), y='Outlays:Q'
)



chart1 = (
    (surplus_area + deficit_area + receipts_line + outlays_line + receipts_label + outlays_label)
    .properties(
        title='U.S. Federal Budget: Surplus (Green) vs. Deficit (Red), 1960–2024',
        width=900,
        height=450
    )
)

chart1


alt.LayerChart(...)

In [32]:

df = pd.read_excel(file_path, sheet_name='Table 1.2')
df['Year'] = pd.to_numeric(df['Years'])
df = df.dropna(subset=['Year'])
df = df[df['Year'] >= 1960]

for col in ['Receipts', 'Outlays', 'Surplus or Deficit (-)']:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace('%', ''))

df_long = df.melt(
    id_vars='Year',
    value_vars=['Receipts', 'Outlays', 'Surplus or Deficit (-)'],
    var_name='Category',
    value_name='Percent_of_GDP'
)

chart = (
    alt.Chart(df_long)
    .mark_line(strokeWidth=2)
    .encode(
        x=alt.X(
            'Year:O',
            title='Fiscal Year',
            axis=alt.Axis(
                labelAngle=90,
                labelFontSize=10,
                titleFontSize=12,
                labelExpr="datum.label % 2 == 0 ? datum.label : ''"
            )
        ),
        y=alt.Y('Percent_of_GDP:Q', title='Percent of GDP'),
        color=alt.Color(
            'Category:N',
            scale=alt.Scale(
                domain=['Receipts', 'Outlays', 'Surplus or Deficit (-)'],
                range=['#1E88E5', '#FB8C00', 'gray']
            ),
            title='Category'
        )
    )
)

zero_line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
    color='black', strokeDash=[2,2]
).encode(y='y:Q')

chart2 = (
    (zero_line + chart)
    .properties(
        title='U.S. Federal Budget as Percent of GDP (1960–2024)',
        width=900,
        height=450
    )
)

chart2


alt.LayerChart(...)

In [33]:
df = pd.read_excel(file_path, sheet_name='Table 1.3')

df['Year'] = pd.to_numeric(df['Years'])
df['Surplus_or_Deficit'] = pd.to_numeric(df['Surplus or Deficit (-)'])
df = df[df['Year'] >= 1940]

df['Surplus'] = df['Surplus_or_Deficit'].where(df['Surplus_or_Deficit'] > 0, 0)
df['Deficit'] = df['Surplus_or_Deficit'].where(df['Surplus_or_Deficit'] < 0, 0)

base = alt.Chart(df).encode(
    x=alt.X(
        'Year:O',
        title='Fiscal Year',
        axis=alt.Axis(
            labelAngle=90,
            labelFontSize=10,
            titleFontSize=12,
            labelExpr="datum.label % 2 == 0 ? datum.label : ''"       #ChatGPT prompt “In Altair, how can I show only every second (or alternate) year label on the x-axis to avoid clutter?”
        )                                                              #Filter to label only alternate years using modulus , labelExpr
    )
)

surplus_area = base.mark_area(color='#4CAF50', opacity=0.5).encode(
    y=alt.Y('Surplus:Q', title='Billions of FY 2017 Dollars'),
    y2=alt.datum(0)
)

deficit_area = base.mark_area(color='#E53935', opacity=0.5).encode(
    y='Deficit:Q',
    y2=alt.datum(0)
)

df_long = df.melt(
    id_vars=['Year'],
    value_vars=['Receipts', 'Outlays'],
    var_name='Category',
    value_name='Value'
)

lines = (
    alt.Chart(df_long)
    .mark_line(strokeWidth=2)
    .encode(
        x='Year:O',
        y='Value:Q',
        color=alt.Color(
            'Category:N',
            scale=alt.Scale(
                domain=['Receipts', 'Outlays'],
                range=['#1E88E5', '#FB8C00']
            ),
            title='Category'
        )
    )
)

chart3 = (
    (surplus_area + deficit_area + lines)
    .properties(
        title='U.S. Federal Budget in Constant (FY 2017) Dollars — Surplus vs. Deficit, 1940–2024',
        width=900,
        height=450
    )
)

chart3

alt.LayerChart(...)

In [34]:
df = pd.read_excel(file_path, sheet_name='Table 1.3')

df['Year'] = pd.to_numeric(df['Years'])
df['Surplus_or_Deficit'] = pd.to_numeric(df['Surplus or Deficit (-)'])

df = df.dropna(subset=['Year'])
df = df[df['Year'] >= 1940]

df_melted = df.melt(id_vars=['Year', 'Surplus_or_Deficit'],
                    value_vars=['Receipts', 'Outlays'],
                    var_name='Category',
                    value_name='Amount')

base = alt.Chart(df_melted).encode(
    x=alt.X('Year:O',
            title='Fiscal Year',
            axis=alt.Axis(labelAngle=90, labelFontSize=10,
                          titleFontSize=12,
                          labelExpr="datum.label % 2 == 0 ? datum.label : ''")),
    y=alt.Y('Amount:Q', title='Billions of FY 2017 Dollars'),
    color=alt.Color('Category:N',
                    scale=alt.Scale(domain=['Receipts', 'Outlays'],
                                    range=['#1E88E5', '#FB8C00']),
                    title='Category'),
    tooltip=['Year', 'Category', 'Amount']
)

stacked_area = base.mark_area(opacity=0.7, interpolate='monotone')

surplus_line = (
    alt.Chart(df)
    .mark_line(color='gray', strokeDash=[4, 4], strokeWidth=2)
    .encode(x='Year:O', y='Surplus_or_Deficit:Q',
            tooltip=['Year', 'Surplus_or_Deficit'])
)

chart3_stacked = (
    (stacked_area + surplus_line)
    .properties(
        title='U.S. Federal Budget (Constant FY 2017 Dollars): Receipts, Outlays & Surplus/Deficit (1940–2024)',
        width=900,
        height=450
    )
)

chart3_stacked


alt.LayerChart(...)

In [35]:
df = pd.read_excel(file_path, sheet_name='Table 2.1')


df_melted = df.melt(                                #ChatGPT prompt: How can I reshape my DataFrame from wide to long format in pandas so that the different revenue sources become a single column?
    id_vars=['Year'],                               #ChatGPT suggested df.melt 
    value_vars=[
        'Individual Income Taxes',
        'Corporation Income Taxes',
        'Social Insurance and Retirement Receipts',
        'Excise Taxes',
        'Other'
    ],
    var_name='Revenue Source',
    value_name='Amount'
)

chart = (
    alt.Chart(df_melted)
    .mark_area(opacity=0.8)
    .encode(
        x=alt.X(
            'Year:O',
            title='Fiscal Year',
            axis=alt.Axis(labelAngle=90, labelExpr="datum.label % 5 == 0 ? datum.label : ''")
        ),
        y=alt.Y('Amount:Q', stack='zero', title='Receipts (Millions of Dollars)'),
        color=alt.Color(
            'Revenue Source:N',
            title='Revenue Source',
            scale=alt.Scale(
                domain=[
                    'Individual Income Taxes',
                    'Corporation Income Taxes',
                    'Social Insurance and Retirement Receipts',
                    'Excise Taxes',
                    'Other'
                ],
                range=['#1E88E5', '#FB8C00', '#43A047', '#8E24AA', '#FDD835']
            )
        ),
        tooltip=['Year', 'Revenue Source', alt.Tooltip('Amount:Q', format=',.0f')]
    )
    .properties(
        title='Composition of Federal Receipts by Source (1934–Present)',
        width=900,
        height=450
    )
)

chart


alt.Chart(...)

In [ ]:
df = pd.read_excel(file_path, sheet_name='Table 2.1')

df_melted = df.melt(
    id_vars=['Year'],
    value_vars=[
        'Individual Income Taxes',
        'Corporation Income Taxes',
        'Social Insurance and Retirement Receipts',
        'Excise Taxes',
        'Other'
    ],
    var_name='Revenue Source',
    value_name='Amount'
)

df_melted['Total'] = df_melted.groupby('Year')['Amount'].transform('sum')
df_melted['Share'] = df_melted['Amount'] / df_melted['Total'] * 100

chart_pct = (
    alt.Chart(df_melted)
    .mark_area(opacity=0.8)
    .encode(
        x=alt.X(
            'Year:O',
            title='Fiscal Year',
            axis=alt.Axis(labelAngle=90, labelExpr="datum.label % 5 == 0 ? datum.label : ''")
        ),
        y=alt.Y('Share:Q', stack='normalize', title='Share of Total Receipts (%)'),
        color=alt.Color(
            'Revenue Source:N',
            title='Revenue Source',
            scale=alt.Scale(
                domain=[
                    'Individual Income Taxes',
                    'Corporation Income Taxes',
                    'Social Insurance and Retirement Receipts',
                    'Excise Taxes',
                    'Other'
                ],
                range=['#1E88E5', '#FB8C00', '#43A047', '#8E24AA', '#FDD835']
            )
        )
    )
    .properties(
        title='Share of Federal Receipts by Source (as % of Total, 1934–Present)',
        width=900,
        height=450
    )
)

chart_pct


alt.Chart(...)

In [46]:
df = pd.read_excel(file_path, sheet_name='Table 2.1')

df_melted = df.melt(
    id_vars=['Year'],
    value_vars=[
        'Individual Income Taxes',
        'Corporation Income Taxes',
        'Social Insurance and Retirement Receipts',
        'Excise Taxes',
        'Other'
    ],
    var_name='Revenue Source',
    value_name='Amount'
)

df_melted['Total'] = df_melted.groupby('Year')['Amount'].transform('sum')
df_melted['Share'] = df_melted['Amount'] / df_melted['Total'] * 100

chart_bar = (
    alt.Chart(df_melted)
    .mark_bar()
    .encode(
        x=alt.X(
            'Year:O',
            title='Fiscal Year',
            axis=alt.Axis(
                labelAngle=90,
                labelExpr="datum.label % 5 == 0 ? datum.label : ''"
            )
        ),
        y=alt.Y(
            'Share:Q',
            stack='normalize',
            title='Share of Total Receipts (%)'
        ),
        color=alt.Color(
            'Revenue Source:N',
            title='Revenue Source',
            scale=alt.Scale(
                domain=[
                    'Individual Income Taxes',
                    'Corporation Income Taxes',
                    'Social Insurance and Retirement Receipts',
                    'Excise Taxes',
                    'Other'
                ],
                range=['#1E88E5', '#FB8C00', '#43A047', '#8E24AA', '#FDD835']
            )
        )
    )
    .properties(
        title='Share of Federal Receipts by Source (100% Stacked Bar, 1934–Present)',
        width=900,
        height=450
    )
)

chart_bar


alt.Chart(...)

In [38]:
df = pd.read_excel(file_path, sheet_name='Table 2.2')

df = df[df['Year'] % 5 == 0]

df_melted = df.melt(
    id_vars=['Year'],
    var_name='Subcategory',
    value_name='Percentage'
)

heatmap = (
    alt.Chart(df_melted)
    .mark_rect()
    .encode(
        x=alt.X('Year:O', title='Year'),
        y=alt.Y('Subcategory:N', title='Category'),
        color=alt.Color(
            'Percentage:Q',
            scale=alt.Scale(scheme='yellowgreenblue'),
            title='Percentage (%)'
        ),
    )
    .properties(
        title='Percentage Contribution of Subcategories Over Time (Every 5 Years)',
        width=800,
        height=250
    )
)

text = (
    heatmap.mark_text(baseline='middle', fontSize=11)
    .encode(
        text=alt.Text('Percentage:Q', format='.1f'),
        color=alt.condition(
            alt.datum.Percentage > 20,
            alt.value('white'),
            alt.value('black')
        )
    )
)

final_chart = (
    (heatmap + text)
    .configure_axis(
        labelFontSize=12,
        titleFontSize=14,
        labelAngle=45
    )
    .configure_title(fontSize=16)
)

final_chart


alt.LayerChart(...)

In [39]:
df = pd.read_excel(file_path, sheet_name='Table 2.4.1')

df.columns = df.columns.str.strip()
df = df.dropna(how='all')

df_long = (
    df.melt(
        id_vars=[df.columns[0]],   
        var_name='Year',
        value_name='Value'
    )
    .rename(columns={df.columns[0]: 'Subcategory'})
)

df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')
df_long['Year'] = pd.to_numeric(df_long['Year'], errors='coerce') 
df_long = df_long.dropna(subset=['Value', 'Year'])

df_long = df_long[df_long['Year'] >= 1960]

chart = (
    alt.Chart(df_long)
    .mark_area(opacity=0.85)
    .encode(
        x=alt.X(
            'Year:O',
            title='Year',
            axis=alt.Axis(labelAngle=90, labelFontSize=10)
        ),
        y=alt.Y(
            'Value:Q',
            stack='zero',  
            title='Value (Millions of Dollars)'
        ),
        color=alt.Color(
            'Subcategory:N',
            title='Subcategory',
            scale=alt.Scale(scheme='tableau20')  
        )

    )
    .properties(
        title='Composition of Social Insurance and Retirement Receipts by Subcategory (1960–Present)',
        width=900,
        height=450
    )

)

chart


alt.Chart(...)

In [40]:
df = pd.read_excel(file_path, sheet_name='Table 2.4.2')

df_long = (
    df.melt(
        id_vars=[df.columns[0]],   
        var_name='Year',
        value_name='Value'
    )
    .rename(columns={df.columns[0]: 'Subcategory'})
)

df_long['Value'] = pd.to_numeric(df_long['Value'])
df_long['Year'] = pd.to_numeric(df_long['Year'])
df_long = df_long.dropna(subset=['Value', 'Year'])

df_long = df_long[df_long['Year'] >= 1960]

chart = (
    alt.Chart(df_long)
    .mark_area(opacity=0.85)
    .encode(
        x=alt.X(
            'Year:O',
            title='Year',
            axis=alt.Axis(labelAngle=90, labelFontSize=10)
        ),
        y=alt.Y(
            'Value:Q',
            stack='zero',  
            title='Value (Millions of Dollars)'
        ),
        color=alt.Color(
            'Subcategory:N',
            title='Subcategory',
            scale=alt.Scale(scheme='tableau20')  
        )
        
    )
    .properties(
        title='Composition of Excise Taxes by Subcategory (1960–Present)',
        width=900,
        height=450
    )

)

chart


alt.Chart(...)

In [41]:
df = pd.read_excel(file_path, sheet_name='Table 3.2')

df_long = df.melt(
    id_vars=[df.columns[0]],  
    var_name='Year',
    value_name='Outlays'
).rename(columns={df.columns[0]: 'Subfunction'})

df_long['Year'] = pd.to_numeric(df_long['Year'])
df_long['Outlays'] = pd.to_numeric(df_long['Outlays'])
df_long = df_long.dropna(subset=['Year', 'Outlays'])

df_long = df_long[df_long['Year'] >= 1960]

chart = (
    alt.Chart(df_long)
    .mark_area(opacity=0.85)
    .encode(
        x=alt.X(
            'Year:O',
            title='Year',
            axis=alt.Axis(labelAngle=90, labelFontSize=10)
        ),
        y=alt.Y(
            'Outlays:Q',
            stack='zero',  
            title='Outlays (Millions of Dollars)'
        ),
        color=alt.Color(
            'Subfunction:N',
            title='Subfunction',
            scale=alt.Scale(scheme='tableau20') 
        ),
        tooltip=[
            alt.Tooltip('Year:O', title='Year'),
            alt.Tooltip('Subfunction:N', title='Subfunction'),
            alt.Tooltip('Outlays:Q', format=',.0f', title='Outlays ($M)')
        ]
    )
    .properties(
        title='Federal Outlays by Subfunction (1960–Present)',
        width=900,
        height=450
    )

)

chart


alt.Chart(...)

In [42]:
df = pd.read_excel(file_path, sheet_name='Table 3.2')

df_long = (
    df.melt(
        id_vars=[df.columns[0], df.columns[1]], 
        var_name='Year',
        value_name='Outlays'
    )
    .rename(columns={df.columns[0]: 'Function', df.columns[1]: 'Subfunction'})
)

df_long['Year'] = pd.to_numeric(df_long['Year'])
df_long['Outlays'] = pd.to_numeric(df_long['Outlays'])
df_long = df_long.dropna(subset=['Year', 'Outlays'])
df_long = df_long[df_long['Year'] >= 1960]
df_long['Outlays'] = df_long['Outlays'].clip(lower=0)

base = (
    alt.Chart(df_long)
    .mark_area(opacity=0.85)
    .encode(
        x=alt.X(
            'Year:O',
            title='Year',
            axis=alt.Axis(labelAngle=90, labelFontSize=9)
        ),
        y=alt.Y(
            'Outlays:Q',
            stack='zero',
            title='Outlays (Millions of Dollars)'
        ),
        color=alt.Color(
            'Subfunction:N',
            title='Subfunction',
            scale=alt.Scale(scheme='tableau20')
        ),

    )
)

facet_chart = base.facet(
    facet='Function:N',
    title='Federal Outlays by Function (1960–Present)',         #ChatGPT prompt In Altair, how can I create small multiples charts to show federal outlays by function, each in its own subplot with independent y-scales
    columns=3                                                   #ChatGPT : You can create small multiples (facets) in Altair using the .facet() method. This allows you to split a chart into multiple subplots (one per category).
).resolve_scale(
    y='independent'
).configure_axis(
    labelFontSize=10,
    titleFontSize=11
).configure_title(
    fontSize=16
)

facet_chart


alt.FacetChart(...)

In [47]:
df = pd.read_excel(file_path, sheet_name='Table 7.1')
df = df[df['Year'] >= 1960]

df[['Year', 'Gross Federal Debt']] = df[['Year', 'Gross Federal Debt']].apply(pd.to_numeric)

df_long = df.melt(
    id_vars='Year',
    value_vars=[
        'Held by Federal Government Accounts',
        'Federal Reserve System',
        'Other'
    ],
    var_name='Debt Component',
    value_name='Amount'
)

bars = alt.Chart(df_long).mark_bar().encode(
    x=alt.X('Year:O', title='Fiscal Year', axis=alt.Axis(labelAngle=90, labelFontSize=9)),
    y=alt.Y('Amount:Q', stack='zero', title='Debt (Millions of Dollars)'),
    color=alt.Color(
        'Debt Component:N',
        title='Debt Holder',
        scale=alt.Scale(
            domain=[
                'Held by Federal Government Accounts',
                'Federal Reserve System',
                'Other'
            ],
            range=['#fdae61', '#4575b4', '#91cf60']
        )
    )
)

line = alt.Chart(df).mark_line(color='black', strokeWidth=2).encode(
    x='Year:O',
    y='Gross Federal Debt:Q'
)

points = alt.Chart(df).mark_point(color='black', filled=True, size=40).encode(
    x='Year:O',
    y='Gross Federal Debt:Q'
)

final_chart = (bars + line + points).properties(
    title='Gross Federal Debt and Its Composition by Holder (1940–Present)',
    width=900,
    height=450
)

final_chart


alt.LayerChart(...)